# Inverse Warp

## 构造坐标矩阵

In [9]:
import torch
from torch.autograd import Variable
import numpy as np

depth = Variable(torch.Tensor(np.ones((8, 64, 64))))
b, h, w = depth.size()
i_range = Variable(torch.arange(0, h).view(1, h, 1).expand(1,h,w)).type_as(depth)  # [1, H, W]
j_range = Variable(torch.arange(0, w).view(1, 1, w).expand(1,h,w)).type_as(depth)  # [1, H, W]
ones = Variable(torch.ones(1,h,w)).type_as(depth)

pixel_coords = torch.stack((j_range, i_range, ones), dim=1)
pixel_coords

Variable containing:
(0 ,0 ,.,.) = 
   0   1   2  ...   61  62  63
   0   1   2  ...   61  62  63
   0   1   2  ...   61  62  63
     ...       ⋱       ...    
   0   1   2  ...   61  62  63
   0   1   2  ...   61  62  63
   0   1   2  ...   61  62  63

(0 ,1 ,.,.) = 
   0   0   0  ...    0   0   0
   1   1   1  ...    1   1   1
   2   2   2  ...    2   2   2
     ...       ⋱       ...    
  61  61  61  ...   61  61  61
  62  62  62  ...   62  62  62
  63  63  63  ...   63  63  63

(0 ,2 ,.,.) = 
   1   1   1  ...    1   1   1
   1   1   1  ...    1   1   1
   1   1   1  ...    1   1   1
     ...       ⋱       ...    
   1   1   1  ...    1   1   1
   1   1   1  ...    1   1   1
   1   1   1  ...    1   1   1
[torch.FloatTensor of size 1x3x64x64]

## Pixel Cam

### pixel_to_cam

In [14]:
import torch
from torch.autograd import Variable
import numpy as np

intrinsics_inv = Variable(torch.Tensor(np.ones((8, 3, 3))))
depth = Variable(torch.Tensor(np.ones((8, 64, 64))))

In [16]:
b, h, w = depth.size()

current_pixel_coords = pixel_coords[:,:,:h,:w].expand(b,3,h,w).contiguous().view(b, 3, -1)  # [B, 3, H*W]
cam_coords = intrinsics_inv.bmm(current_pixel_coords).view(b, 3, h, w)
cam_coords * depth.unsqueeze(1)

Variable containing:
(0 ,0 ,.,.) = 
    1    2    3  ...    62   63   64
    2    3    4  ...    63   64   65
    3    4    5  ...    64   65   66
      ...         ⋱        ...      
   62   63   64  ...   123  124  125
   63   64   65  ...   124  125  126
   64   65   66  ...   125  126  127

(0 ,1 ,.,.) = 
    1    2    3  ...    62   63   64
    2    3    4  ...    63   64   65
    3    4    5  ...    64   65   66
      ...         ⋱        ...      
   62   63   64  ...   123  124  125
   63   64   65  ...   124  125  126
   64   65   66  ...   125  126  127

(0 ,2 ,.,.) = 
    1    2    3  ...    62   63   64
    2    3    4  ...    63   64   65
    3    4    5  ...    64   65   66
      ...         ⋱        ...      
   62   63   64  ...   123  124  125
   63   64   65  ...   124  125  126
   64   65   66  ...   125  126  127
     ⋮ 

(1 ,0 ,.,.) = 
    1    2    3  ...    62   63   64
    2    3    4  ...    63   64   65
    3    4    5  ...    64   65   66
      ...         ⋱ 

### cam_to_pixel

In [ ]:
import torch
from torch.autograd import Variable
import numpy as np

cam_coords
proj_c2p_rot
proj_c2p_tr
padding_mode = 'zeros'

## 构造旋转矩阵
四元数(quaternion)和欧拉坐标(Eluer)是空间运动的两种表示方式

### quat_to_mat

In [17]:
import torch
from torch.autograd import Variable
import numpy as np

quat = Variable(torch.Tensor(np.ones((8, 3))))

In [18]:
norm_quat = torch.cat([quat[:,:1].detach()*0 + 1, quat], dim=1)
norm_quat = norm_quat/norm_quat.norm(p=2, dim=1, keepdim=True)
w, x, y, z = norm_quat[:,0], norm_quat[:,1], norm_quat[:,2], norm_quat[:,3]

B = quat.size(0)

w2, x2, y2, z2 = w.pow(2), x.pow(2), y.pow(2), z.pow(2)
wx, wy, wz = w*x, w*y, w*z
xy, xz, yz = x*y, x*z, y*z

rotMat = torch.stack([w2 + x2 - y2 - z2, 2*xy - 2*wz, 2*wy + 2*xz,
                      2*wz + 2*xy, w2 - x2 + y2 - z2, 2*yz - 2*wx,
                      2*xz - 2*wy, 2*wx + 2*yz, w2 - x2 - y2 + z2], dim=1).view(B, 3, 3)
rotMat

Variable containing:
(0 ,.,.) = 
  0  0  1
  1  0  0
  0  1  0

(1 ,.,.) = 
  0  0  1
  1  0  0
  0  1  0

(2 ,.,.) = 
  0  0  1
  1  0  0
  0  1  0

(3 ,.,.) = 
  0  0  1
  1  0  0
  0  1  0

(4 ,.,.) = 
  0  0  1
  1  0  0
  0  1  0

(5 ,.,.) = 
  0  0  1
  1  0  0
  0  1  0

(6 ,.,.) = 
  0  0  1
  1  0  0
  0  1  0

(7 ,.,.) = 
  0  0  1
  1  0  0
  0  1  0
[torch.FloatTensor of size 8x3x3]

### eluer_to_mat

In [22]:
import torch
from torch.autograd import Variable
import numpy as np

angle = Variable(torch.Tensor(np.ones((8, 3))))

In [23]:
B = angle.size(0)
x, y, z = angle[:,0], angle[:,1], angle[:,2]

cosz = torch.cos(z)
sinz = torch.sin(z)

zeros = z.detach() * 0
ones = zeros.detach()+1
zmat = torch.stack([cosz, -sinz, zeros,
                    sinz,  cosz, zeros,
                    zeros, zeros,  ones], dim=1).view(B, 3, 3)

cosy = torch.cos(y)
siny = torch.sin(y)

ymat = torch.stack([cosy, zeros,  siny,
                    zeros,  ones, zeros,
                    -siny, zeros,  cosy], dim=1).view(B, 3, 3)

cosx = torch.cos(x)
sinx = torch.sin(x)

xmat = torch.stack([ones, zeros, zeros,
                    zeros,  cosx, -sinx,
                    zeros,  sinx,  cosx], dim=1).view(B, 3, 3)

rotMat = xmat.bmm(ymat).bmm(zmat)
rotMat

Variable containing:
(0 ,.,.) = 
  0.2919 -0.4546  0.8415
  0.8372 -0.3039 -0.4546
  0.4624  0.8372  0.2919

(1 ,.,.) = 
  0.2919 -0.4546  0.8415
  0.8372 -0.3039 -0.4546
  0.4624  0.8372  0.2919

(2 ,.,.) = 
  0.2919 -0.4546  0.8415
  0.8372 -0.3039 -0.4546
  0.4624  0.8372  0.2919

(3 ,.,.) = 
  0.2919 -0.4546  0.8415
  0.8372 -0.3039 -0.4546
  0.4624  0.8372  0.2919

(4 ,.,.) = 
  0.2919 -0.4546  0.8415
  0.8372 -0.3039 -0.4546
  0.4624  0.8372  0.2919

(5 ,.,.) = 
  0.2919 -0.4546  0.8415
  0.8372 -0.3039 -0.4546
  0.4624  0.8372  0.2919

(6 ,.,.) = 
  0.2919 -0.4546  0.8415
  0.8372 -0.3039 -0.4546
  0.4624  0.8372  0.2919

(7 ,.,.) = 
  0.2919 -0.4546  0.8415
  0.8372 -0.3039 -0.4546
  0.4624  0.8372  0.2919
[torch.FloatTensor of size 8x3x3]

## inverse_warp

In [ ]:
img = 0
depth = 0
pose = 0
intrinsics = 0
intrinsics_inv = 0
rotation_mode = 0
padding_mode = 0